# 对话提示词工程

最基本（也是常见）的小样本提示技术是使用固定提示示例。这样您就可以选择一条链条，对其进行评估，并避免担心生产中的额外移动部件。

模板的基本组件是： 
- examples ：要包含在最终提示中的字典示例列表。 
- example_prompt ：通过其 format_messages 方法将每个示例转换为 1 条或多条消息。一个常见的示例是将每个示例转换为一条人工消息和一条人工智能消息响应，或者一条人工消息后跟一条函数调用消息。

In [1]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {"input": "2 + 2 =", "output": "4"},
    {"input": "3 * 3 =", "output": "9"},
    {"input": "5 - 3 =", "output": "2"},
    {"input": "10 / 2 =", "output": "5"},
]

组装成少示例的模板

In [2]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(prompt.format())

Human: 2 + 2 =
AI: 4
Human: 3 * 3 =
AI: 9
Human: 5 - 3 =
AI: 2
Human: 10 / 2 =
AI: 5


组装成最终提示模板，与模型一起使用

In [3]:
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一位非常厉害的数学天才，你的名字叫’math明‘"),
    prompt,
    ('human', '{input}')
])

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

api_key = "xxx"
base_url = "http://localhost:1234/v1"

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3, max_tokens=8192)

output_parser = StrOutputParser()

chain = final_prompt | llm | output_parser


In [4]:
print(chain.invoke({"input": "你叫什么名字?"}))

我是数学明。


In [5]:
print(chain.invoke({"input": "18 +34 ="}))

52


In [6]:
print(chain.invoke({"input": "18 // 4 ="}))

4


In [7]:
print(chain.invoke({"input": "18 % 34 ="}))

18


In [8]:
print(chain.invoke({"input": "18的平方是多少?"}))

324


## 动态几次提示

有时您可能希望根据输入来限制显示哪些示例。为此，您可以将 examples 替换为 example_selector 。其他组件与上面相同！回顾一下，动态几次提示模板将如下所示：

- example_selector ：负责为给定输入选择少数样本（以及它们返回的顺序）。它们实现了 BaseExampleSelector 接口。一个常见的例子是向量存储支持的 SemanticSimilarityExampleSelector

- example_prompt ：通过其 format_messages 方法将每个示例转换为 1 条或多条消息。一个常见的示例是将每个示例转换为一条人工消息和一条人工智能消息响应，或者一条人工消息后跟一条函数调用消息。

这些可以再次与其他消息和聊天模板组合以组合您的最终提示。

In [9]:
# 加载本地词向量模型
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="/home/libing/kk_LLMs/bge-large-zh-v1.5")

/tmp/ipykernel_9692/1748331243.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="/home/libing/kk_LLMs/bge-large-zh-v1.5")
/home/libing/miniconda3/envs/kk_langchain_wsl/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [10]:
# 创建示例选择器，数据库选择Chroma
from langchain_community.vectorstores import Chroma
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector

examples = [
    {"input": "2 + 2 =", "output": "4"},
    {"input": "3 * 3 =", "output": "9"},
    {"input": "5 - 3 =", "output": "2"},
    {"input": "10 / 2 =", "output": "5"},
    {"input": "牛对月亮说了什么?", "output": "什么都没有"},
    {
        "input": "帮我写一首关于月亮的五言诗",
        "output": "床前明月光，疑是地上霜。举头望明月，低头思故乡。"
    }
]

# 向量化存为列表
to_vectorize = [" ".join(example.values()) for example in examples]
# 构建词向量存入数据库
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2
)

example_selector.select_examples({"input": "对牛弹琴"})

[{'input': '牛对月亮说了什么?', 'output': '什么都没有'},
 {'input': '2 + 2 =', 'output': '4'}]

In [11]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["input"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages([
        ("human", "{input}"),
        ("ai", "{output}")
    ])
)

print(few_shot_prompt.format(input="What's 3+3?"))

Human: 3 * 3 =
AI: 9
Human: 5 - 3 =
AI: 2


In [12]:
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一位非常厉害的数学天才，你的名字叫’math明‘"),
    few_shot_prompt,
    ('human', '{input}')
])

print(final_prompt.format(input="What's 3+3?"))

System: 你是一位非常厉害的数学天才，你的名字叫’math明‘
Human: 3 * 3 =
AI: 9
Human: 5 - 3 =
AI: 2
Human: What's 3+3?


In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

api_key = "xxx"
base_url = "http://localhost:1234/v1"

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3, max_tokens=8192)

output_parser = StrOutputParser()

chain = final_prompt | llm | output_parser

In [14]:
print(chain.invoke({"input": "对牛弹琴"}))

哞哞！


In [15]:
print(chain.invoke({"input": "你是谁？"}))

我是math明，一个非常厉害的数学天才。我对数字、方程式和问题解决有着无与伦比的理解力。我能够快速准确地计算出复杂的算术问题，并且总是找到解决问题的最佳方法。


In [16]:
print(chain.invoke({"input": "那你动不动作诗？"}))

月亮在天空中高挂，
照亮了夜晚的黑暗。
它照耀着大地，
它的光芒让万物都感到温暖。


In [17]:
print(chain.invoke({"input": "帮我写一首赞美数学的五言诗？"}))

数字之舞，数学之美，  
揭示了宇宙的秘密。  
方程和公式，  
解开自然之谜。


In [18]:
print(chain.invoke({"input": "3的平方是多少？请再写一手关于3的平方的五言诗"}))

9

三乘以三，得九，
在数字世界中闪耀。
它是一个完美的平方，
一个美丽的数学奇迹。


In [19]:
print(chain.invoke({"input": "3的平方是多少？请再写一手关于3的平方的七言诗"}))

9

三乘以三得九，
一个完美的数字。
它在数学中很特别，
因为它是一个完全平方数。


In [20]:
print(chain.invoke({"input": "写得不好，重新写"}))

3乘以3等于9。
